In [1]:
import pandas as pd
import re
from collections import Counter
#pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def assignDivision(divisionsDict, divisions):
    for key in divisionsDict:
        div = re.split(r' |/|\+|,|-', divisionsDict[key])
        for d in divisions: #loop through list with known divisions
            if d in div:
                assignedDiv = d
                break
        divisionDict[key] = {'Assigned division': assignedDiv}
    return divisionDict

In [24]:
#load 
researchers = pd.read_pickle("Files/Dataframes/researchers_wGender.pkl").reset_index()
researchers['Division'] = researchers.Division.fillna('Unknown')

In [25]:
researchers['AllDivisions'] = researchers.Division.str.split(r' |/|\+|,|-')
researchers.explode('AllDivisions') 

,index,ID,OriginalName,API_Name,AuthorCount,MaxPaperCount,Field,FirstPaperYear,Year,Name,...,Title,Division,SessionType,SessionIndex,FirstName,Seniority,CumulativePaperCount,Productivity,Gender,AllDivisions
0,0,155004,Cong Meng,C. Meng,93,1023,Physics,1992,2013,Cong Meng,...,Self-Organized Growth of Single Crystalline Co...,DCMP,participant,122791.0,Cong,21,137,6.523810,M,DCMP
1,1,155007,Kris Kempa,K. Kempa,1,75,Physics,1987,2012,Kris Kempa,...,Plasmonic nanowire transmision lines,DCMP,participant,122798.0,Kris,25,63,2.520000,M,DCMP
2,2,155007,Kris Kempa,K. Kempa,1,75,Physics,1987,2003,Kris Kempa,...,Electronic Structure of Carbon Nanotube Peapods,DMP/FIAP,participant,635310.0,Kris,16,30,1.875000,M,DMP
2,2,155007,Kris Kempa,K. Kempa,1,75,Physics,1987,2003,Kris Kempa,...,Electronic Structure of Carbon Nanotube Peapods,DMP/FIAP,participant,635310.0,Kris,16,30,1.875000,M,FIAP
3,3,155014,Cedric Vaillant,C. Vaillant,4,93,Physics,1985,2022,Cedric Vaillant,...,How to correctly simulate local chromosome org...,None,participant,122816.0,Cedric,37,83,2.243243,M,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146440,160231,35999,Yue Chen,Yue Chen,2296,2654,Physics,1972,2018,Yue Chen,...,A first-principles study of the anharmonic lat...,GSCCM DMP DCOMP,participant,114409.0,Yue,46,1598,34.739130,M,DCOMP
146441,160232,35999,Yue Chen,Yue Chen,2296,2654,Physics,1972,2013,Yue Chen,...,Anomalous Anharmonic Phonons in PbTe Reproduce...,DCOMP,participant,123275.0,Yue,41,915,22.317073,M,DCOMP
146442,160233,35999,Yue Chen,Yue Chen,2296,2654,Physics,1972,2021,Yue Chen,...,Pressure-induced metal-insulator transition in...,DMP DCOMP GMAG,participant,133532.0,Yue,49,2138,43.632653,M,DMP
146442,160233,35999,Yue Chen,Yue Chen,2296,2654,Physics,1972,2021,Yue Chen,...,Pressure-induced metal-insulator transition in...,DMP DCOMP GMAG,participant,133532.0,Yue,49,2138,43.632653,M,DCOMP


In [4]:
# opening the file in read mode 
sponsorFile = open("Files/sponsoringUnits.txt", "r").read()
sponsorList = sponsorFile.replace('\n', ';').split(';')
sponsor_abr = re.findall(r'\((.*)\)', sponsorFile) #abbreviations for the sponsoring units
sponsorNames = re.findall(r'(.*) \(', sponsorFile) #ful name of sponsoring unit

In [5]:
#count the number of sessions within each division
division = researchers.Division.to_list()
divisions = [w for d in division for w in re.split(r' |/|\+|,|-', d)] 
divisionCount = dict(Counter(divisions))
uniqueDivisions = list(divisionCount.keys()) #unique sponsoring units

In [6]:
#sorted divisions in increasing order
sortedDivisions = sorted(divisionCount, key=divisionCount.get, reverse=True)

In [7]:
#known divisions in data, in order of size according to divisionCount
knownDivisions = [d for d in sortedDivisions if d in sponsor_abr]

In [8]:
divisionDict = dict(zip(researchers.index, researchers.Division))
assignedDivisions = assignDivision(divisionDict, knownDivisions)

In [9]:
division_df = pd.DataFrame.from_dict(assignedDivisions, orient='index').rename(columns={"Assigned division": "AssignedDivision"})

In [10]:
#add assigned division
researchers = pd.merge(researchers, division_df, left_index=True, right_index=True)

In [11]:
researchers.to_pickle("Files/Dataframes/researchersFinal.pkl") 